# **Planning**

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from typing import Literal
from queue import Queue, LifoQueue, PriorityQueue

print(os.getcwd())
os.chdir("../utils/")
sys.path.append("../utils/planning.py")

from planning import valid_actions, visualize_path

/Users/ninovation/Projects/Software/SourceCode/Aeternalis-Ingenium/Drone-Flight-Planning/src/notebooks


## **State Space**

![grid-state-space](https://raw.githubusercontent.com/Aeternalis-Ingenium/Drone-Flight-Planning/trunk/assets/pln/grid-search0.jpg)

In [2]:
grid = np.array(
    [
        [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    ]
)
start = (0, 0)
goal = (6, 10)


## **Breadth-First Search**

In [3]:
def bfs(grid, start_node, goal_node):
    """
    Perform breadth-first search on the grid to find a path from start to goal.
    """

    path = []
    path_cost = 0
    total_exploration_cost = 0
    queue = Queue()
    queue.put((0, start_node))
    visited = set(start_node)

    branch = {}
    found = False

    while not queue.empty():
        current_cost, current_node = queue.get()
        total_exploration_cost += current_cost 

        if current_node == goal_node:        
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_node):
                da = action.value
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                branch_cost = current_cost + 1

                if next_node not in visited:                
                    visited.add(next_node)               
                    branch[next_node] = (branch_cost, current_node, action)
                    queue.put((branch_cost, next_node))

    if found:
        n = goal_node
        path_cost = branch[n][0]
        while branch[n][1] != start_node:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
    else:
        print("######################")
        print("Failed to find a path!")
        print("######################")

    return path[::-1], path_cost, total_exploration_cost

In [4]:
path_bfs, cost_bfs, total_exploration_cost_bfs = bfs(grid, start, goal)

print("BFS Path:", path_bfs)
print("BFS Path Cost:", cost_bfs)
print("BFS Total Exploration Cost:", total_exploration_cost_bfs)

Found a path.
BFS Path: [<Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.UP: (-1, 0, 1)>]
BFS Path Cost: 24
BFS Total Exploration Cost: 1257


In [5]:
visualize_path(grid, path_bfs, start)

array([['S', 'O', ' ', ' ', ' ', 'O', ' ', 'O', ' ', 'O', ' '],
       ['v', 'O', ' ', 'O', ' ', ' ', ' ', 'O', ' ', 'O', ' '],
       ['v', ' ', 'O', ' ', ' ', 'O', ' ', 'O', ' ', ' ', ' '],
       ['v', ' ', 'O', '>', '>', '>', 'v', 'O', ' ', 'O', ' '],
       ['v', 'O', '>', '^', ' ', 'O', 'v', 'O', ' ', 'O', ' '],
       ['>', '>', '^', 'O', ' ', 'O', '>', '>', 'v', 'O', ' '],
       ['O', 'O', ' ', 'O', ' ', 'O', ' ', 'O', 'v', 'O', 'G'],
       [' ', ' ', ' ', 'O', ' ', ' ', 'O', ' ', 'v', 'O', '^'],
       [' ', ' ', ' ', ' ', ' ', 'O', ' ', 'O', '>', '>', '^'],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', 'O', ' ', 'O', ' '],
       [' ', ' ', ' ', 'O', ' ', 'O', ' ', 'O', ' ', 'O', ' '],
       [' ', ' ', ' ', ' ', ' ', 'O', ' ', ' ', ' ', 'O', ' ']],
      dtype='<U1')

## **Depth-First Search**

In [6]:
def dfs(grid, start_node, goal_node):
    """
    Perform depth-first search on the grid to find a path from start to goal.
    """

    path = []
    path_cost = 0
    total_exploration_cost = 0
    stack = LifoQueue()
    stack.put((0, start_node))
    visited = set(start_node)

    branch = {}
    found = False

    while not stack.empty():
        current_cost, current_node = stack.get()
        total_exploration_cost += current_cost 

        if current_node == goal_node:        
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_node):
                da = action.value
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                branch_cost = current_cost + 1

                if next_node not in visited:
                    visited.add(next_node)
                    branch[next_node] = (branch_cost, current_node, action)
                    stack.put((branch_cost, next_node))

    if found:
        n = goal_node
        path_cost = branch[n][0]
        while branch[n][1] != start_node:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
    else:
        print("######################")
        print("Failed to find a path!")
        print("######################")
            
    return path[::-1], path_cost, total_exploration_cost

In [7]:
path_dfs, cost_dfs, total_exploration_cost_dfs = bfs(grid, start, goal)

print("DFS Path:", path_dfs)
print("DFS Path Cost:", cost_dfs)
print("DFS Total Exploration Cost:", total_exploration_cost_dfs)

Found a path.
DFS Path: [<Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.UP: (-1, 0, 1)>]
DFS Path Cost: 24
DFS Total Exploration Cost: 1257


In [8]:
visualize_path(grid, path_dfs, start)

array([['S', 'O', ' ', ' ', ' ', 'O', ' ', 'O', ' ', 'O', ' '],
       ['v', 'O', ' ', 'O', ' ', ' ', ' ', 'O', ' ', 'O', ' '],
       ['v', ' ', 'O', ' ', ' ', 'O', ' ', 'O', ' ', ' ', ' '],
       ['v', ' ', 'O', '>', '>', '>', 'v', 'O', ' ', 'O', ' '],
       ['v', 'O', '>', '^', ' ', 'O', 'v', 'O', ' ', 'O', ' '],
       ['>', '>', '^', 'O', ' ', 'O', '>', '>', 'v', 'O', ' '],
       ['O', 'O', ' ', 'O', ' ', 'O', ' ', 'O', 'v', 'O', 'G'],
       [' ', ' ', ' ', 'O', ' ', ' ', 'O', ' ', 'v', 'O', '^'],
       [' ', ' ', ' ', ' ', ' ', 'O', ' ', 'O', '>', '>', '^'],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', 'O', ' ', 'O', ' '],
       [' ', ' ', ' ', 'O', ' ', 'O', ' ', 'O', ' ', 'O', ' '],
       [' ', ' ', ' ', ' ', ' ', 'O', ' ', ' ', ' ', 'O', ' ']],
      dtype='<U1')

## **Heuristic Search**

In [ ]:
def heuristic_a(next_node, goal_node):
    return np.sqrt((next_node[0] - goal_node[0])**2 + (next_node[1] - goal_node[1])**2)


def heuristic_b(next_node, goal_node):
    return (next_node[0] - goal_node[0]) + (next_node[1] - goal_node[1])


def h(htype: Literal["a", "b"], next_node, goal_node):
    if htype == "a":
        return heuristic_a(next_node=next_node, goal_node=goal_node)
    return heuristic_b(next_node=next_node, goal_node=goal_node)

## **$A^{*}$**

In [10]:
def a_star(grid, htype, start_node, goal_node):

    path = []
    path_cost = 0
    total_exploration_cost = 0
    queue = PriorityQueue()
    queue.put((0, start_node))
    visited = set(start_node)

    branch = {}
    found = False
    
    while not queue.empty():
        current_cost, current_node = queue.get()
        total_exploration_cost += current_cost
            
        if current_node == goal_node:
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_node):
                da = action.delta
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                branch_cost = current_cost + action.cost
                queue_cost = branch_cost + h(htype=htype, next_node=next_node, goal_node=goal_node)

                if next_node not in visited:                
                    visited.add(next_node)               
                    branch[next_node] = (branch_cost, current_node, action)
                    queue.put((queue_cost, next_node))

    if found:
        n = goal_node
        path_cost = branch[n][0]
        while branch[n][1] != start_node:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
    else:
        print("######################")
        print("Failed to find a path!")
        print("######################")

    return path[::-1], path_cost, total_exploration_cost

In [14]:
path_as, cost_as, total_exploration_cost_as = a_star(grid, "a", start, goal)

print("A* Path:", path_as)
print("A* Path Cost:", cost_as)
print("A* Total Exploration Cost:", total_exploration_cost_as)

Found a path.
A* Path: [<Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1)>, <Action.UP: (-1, 0, 1)>]
A* Path Cost: 159.4407161316494
A* Total Exploration Cost: 9210.33033098142


In [15]:
visualize_path(grid, path_as, start)

array([['S', 'O', ' ', ' ', ' ', 'O', ' ', 'O', ' ', 'O', ' '],
       ['v', 'O', ' ', 'O', ' ', ' ', ' ', 'O', ' ', 'O', ' '],
       ['v', ' ', 'O', ' ', ' ', 'O', ' ', 'O', ' ', ' ', ' '],
       ['v', ' ', 'O', ' ', '>', '>', 'v', 'O', ' ', 'O', ' '],
       ['v', 'O', '>', '>', '^', 'O', 'v', 'O', ' ', 'O', ' '],
       ['>', '>', '^', 'O', ' ', 'O', '>', '>', 'v', 'O', ' '],
       ['O', 'O', ' ', 'O', ' ', 'O', ' ', 'O', 'v', 'O', 'G'],
       [' ', ' ', ' ', 'O', ' ', ' ', 'O', ' ', 'v', 'O', '^'],
       [' ', ' ', ' ', ' ', ' ', 'O', ' ', 'O', '>', '>', '^'],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', 'O', ' ', 'O', ' '],
       [' ', ' ', ' ', 'O', ' ', 'O', ' ', 'O', ' ', 'O', ' '],
       [' ', ' ', ' ', ' ', ' ', 'O', ' ', ' ', ' ', 'O', ' ']],
      dtype='<U1')